# Experiment 5: Implementation of FP-Growth Algorithm

### Steps:
#### Collect : Any method
#### Prepare: Dicrete data needed
#### Analyze: Any method
#### Train: Build an FP-Tree and mine it
#### Test: Doesn't apply
#### Use: identify commonly occuring items in order to make decision

In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    # increamenting the count    
    def inc(self, numOccur):
        self.count += numOccur
        
    #display tree text
    def disp(self, ind=1):
        print(' '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

## Test the class implementation

### Create the root node

In [2]:
rootNode = treeNode('pyramid', 9, None)

### Add child to the root node

In [3]:
rootNode.children['eye'] = treeNode('eye', 13, None)

In [4]:
rootNode.disp()

  pyramid   9
   eye   13


## Construct the FP-Tree

### createTree()

In [5]:
def createTree(dataSet, minSup=1):
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):
        if headerTable[k] < minSup:
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None)
    for transSet, count in dataSet.items():
        localD = {}
        for item in transSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable            

## updateTree()

In [6]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

### updateHeader()

In [7]:
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

## loadSimpDat()

In [8]:
def loadSimpDat():
    simpDat = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
              ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
              ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
              ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
              ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]
    return simpDat

### createInitSet()

In [9]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [10]:
simpDat = loadSimpDat()

In [11]:
simpDat

[['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
 ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
 ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

In [12]:
initSet = createInitSet(simpDat)

In [13]:
initSet

{frozenset({'Eggs', 'Kidney Beans', 'Milk', 'Nutmeg', 'Onion', 'Yogurt'}): 1,
 frozenset({'Dill', 'Eggs', 'Kidney Beans', 'Nutmeg', 'Onion', 'Yogurt'}): 1,
 frozenset({'Apple', 'Eggs', 'Kidney Beans', 'Milk'}): 1,
 frozenset({'Corn', 'Kidney Beans', 'Milk', 'Unicorn', 'Yogurt'}): 1,
 frozenset({'Corn', 'Eggs', 'Ice cream', 'Kidney Beans', 'Onion'}): 1}

### FP Tree

In [14]:
myFPtree, myHeaderTab = createTree(initSet, 3)

In [15]:
myFPtree.disp()

  Null Set   1
   Kidney Beans   5
    Eggs   4
     Onion   3
      Yogurt   2
       Milk   1
     Milk   1
    Yogurt   1
     Milk   1


### Mining frequent items from the FP tree

### ascendTree()

In [16]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

### findPrefixPath()

In [17]:
def findPrefixPath(basePat, treeNode):
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [23]:
findPrefixPath('X', myHeaderTab['Milk'][1])

{frozenset({'Eggs', 'Kidney Beans', 'Onion', 'Yogurt'}): 1,
 frozenset({'Eggs', 'Kidney Beans'}): 1,
 frozenset({'Kidney Beans', 'Yogurt'}): 1}

In [22]:
findPrefixPath('r', myHeaderTab['Yogurt'][1])

{frozenset({'Eggs', 'Kidney Beans', 'Onion'}): 2,
 frozenset({'Kidney Beans'}): 1}